In [228]:
import pandas as pd
import numpy as np
import chardet
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [229]:
df = pd.read_csv("ecom_yl.csv", sep=",", decimal='.')

In [230]:
#приводим названия столбцов в формат PEP8

df.columns = df.columns.str.lower().str.replace(' ', '_')

In [231]:
def time(row):
    times = {
        'morning': [datetime.time(hour=6), datetime.time(hour=9, minute=59)],
        'day': [datetime.time(hour=10), datetime.time(hour=16, minute=59)],
        'evening': [datetime.time(hour=17), datetime.time(hour=21, minute=59)],
        'night': [datetime.time(hour=22), datetime.time(hour=5, minute=59)]
    }
    a = row.session_start.time
    b = row.session_end.time
    for i in times.keys:
        if a >= times[i][0] and b <= times[i][1]:
            return i
        if a >= times[i][0] and b > times[i][1]:
            if row.sessiondurationsec / 2 <= (times[i][1] - a).seconds:
                return i
        if a <= times[i][0] and b >= times[i][1]:
            return i
        if a < times[i][0] and b <= times[i][1]:
            if row.sessiondurationsec / 2 <= (b - times[i][0]).seconds:
                return i

In [232]:
#исправляем неправильные названия в region

dict_regions = {'United States':'United States', 'Frаnce':'France', 'Unjted States' : 'United States', 'Germany' : 'Germany', 
               'UK' : 'UK', 'France':'France', 'Frаncе': 'France', 'germany':'Germany', 'UК':'UK'}
df["region"] = df["region"].map(dict_regions)

In [233]:
#исправляем неправильные названия в channel

dict_channels = {'социальные сети' : 'социальные сети', 'organic':'organic', 'реклама у блогеров':'реклама у блогеров', 
                 'контексная реклама' : 'контекстная реклама', 'контекстная реклама':'контекстная реклама', 'email-рассылки':'email-рассылки'}
df["channel"] = df["channel"].map(dict_channels)

In [234]:
#исправляем неправильные названия в device

dict_devices = {'iPhone' : 'iPhone', 'PC' : 'PC', 'Mac' : 'Mac', 'Android' : 'Android', 'android' : 'Android'}
df["device"] = df["device"].map(dict_devices)

In [235]:
#убираем выбросы у SessionDuration с помощью 3-ного интерквартильного размаха

def transformed(x):
    i_quant = x.quantile(0.75) - x.quantile(0.25)
    k = x > (x.quantile(0.75) + i_quant * 3)
    x[k] = x.median()
    return x
    
df["sessiondurationsec"] = df.groupby("channel", group_keys=False)["sessiondurationsec"].transform(transformed)
df['revenue'] = df['revenue'].apply(lambda x: 4999 if (x == 100000) or (x == 1) else x)

In [236]:
#переводим даты из строкого формата в datetime

df['session_start'] = pd.to_datetime(df['session_start'])
df['session_end'] = pd.to_datetime(df['session_end'])

In [237]:
#заполняем пропуски в SessionDurationSec

df['sessiondurationsec'] = (df['session_end'] - df['session_start']).dt.total_seconds()

In [238]:
#Удаляем сессии длящиеся 5 секунд и меньше, в которых не было покупок

df = df[(df["sessiondurationsec"] > 5) | ((df["sessiondurationsec"] <= 5) & (df['revenue'].notna()))]

In [239]:
# число явных дубликатов и их удаление
i_see_it = len(df) - len(df.drop_duplicates())
#print(i_see_it) -> 3
df = df.drop_duplicates()

In [240]:
# заполнение пропусков
stupid_nans = df.groupby(['month'])['region']
df['region'] = df['region'].fillna(lambda x: stupid_nans[x])
stupid_nans = df.groupby(['month'])['device']
df['device'] = df['device'].fillna(lambda x: stupid_nans[x])
stupid_nans = df.groupby(['month'])['channel']
df['channel'] = df['channel'].fillna(lambda x: stupid_nans[x])

In [241]:
#очищение неправильных данных в promo_code
df['promo_code'] = df['promo_code'].map(lambda x: 1 if not pd.isna(x) and x else x)

In [242]:
#Добавление итоговой суммы покупки с учетом использования промокода
df['revenue_with_promo_code'] = df.apply(lambda x: 0.9 * x.revenue if x.promo_code == 1 and not pd.isna(x.revenue) else x.revenue, axis=1)

In [243]:
#Обозначение платящих пользователей
df['payer'] = df['revenue'].apply(lambda x: 1 if not pd.isna(x) else 0)

In [ ]:
df['time_of_day'] = df.apply(time, axis=1)

In [ ]:
df

In [222]:
df.to_csv("res.csv", index=False)

In [223]:
df = pd.read_csv("res.csv", sep=",", decimal='.')